<a href="https://colab.research.google.com/github/Kunal02880/Qunatum-phenomeno-/blob/main/Rabi_oscillations_and_Ramsey_spectroscopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [ ]:


def Ham(omega,delta):
  Hint = Qobj([[0,omega],[omega,0]])
  H0 = Qobj([[delta,0],[0,delta]])
  return Hint+H0
def Ham_t(t,**kwargs):
  omega = kwargs["omega"]
  delta = kwargs["delta"]
  Hint = Qobj([[0,omega],[omega,0]])
  H0 = Qobj([[delta,0],[0,delta]])
  if t<75:
    Hret = Hint+H0
  else:
    Hret =  H0
  return Hret

omega = 1
argmap = {"omega":omega,"delta":0.0}

t = np.linspace(0,100,100)
pulse = t*0
ind = np.where(t<75)
pulse[ind[0]] = 1


rh0 = basis(2,0)*basis(2,0).dag()
rh1 = basis(2,1)*basis(2,1).dag()
e_ops = [sigmax(),sigmaz(),sigmay(),rh0,rh1]
options = Options(store_states=True)
results = mesolve(Ham_t,fock(2,0),t,e_ops, args = argmap) # Assuming mesolve is defined elsewhere
fig, ax = plt.subplots()
ax.plot(t,results.expect[3]);
ax.plot(t,results.expect[4]);
ax.plot(t,pulse);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho0", "rho1", "pulse"))

ax.plot(t,results.expect[4]);
ax.plot(t,pulse);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho0", "rho1", "pulse"))



In [ ]:
def Ham(Delta, Omega):
    # laser interaction
    Hint = Qobj([[0, Omega], [Omega, 0]] )
    # laser detuning
    H0 = Qobj([[Delta, 0], [0, 0]] )
    return H0+Hint

In [ ]:
def Ham_t(t, args):
    Omega = args['Omega']
    Delta = args['Delta']
    # laser interaction
    Hint = Qobj([[0, Omega], [Omega, 0]] )
    # laser detuning
    H0 = Qobj([[Delta, 0], [0, 0]] )

    if(t<75):
        Hret = H0+Hint
    else:
        Hret = H0


    return Hret


In [ ]:
# Test of time dependent Hamiltonian

Omega = 0.01
argmap = {'Omega': Omega, 'Delta': 0.0}

rho0 = fock_dm(2,0)
rho1 = fock_dm(2,1)

# time to simulate
t = np.linspace(0, 100, 100)
pulse = 0*t
ind = np.where(t<75)
pulse[ind[0]] = 1


# operators we would like to know the expectation value
e_ops = [sigmax(), sigmay(), sigmaz(), rho0, rho1]

# Shroedinger Equation solver
results = sesolve(Ham_t, fock(2,0), t, e_ops, args = argmap)

fig, ax = plt.subplots()
ax.plot(t,results.expect[3]);
ax.plot(t,results.expect[4]);
ax.plot(t,pulse);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho0", "rho1", "pulse"))

# Showing on the Bloch sphere
b = Bloch()
pnt = [results.expect[0],results.expect[1],results.expect[2]]
b.add_points(pnt)
b.show()

In [ ]:
# Rabi frequency
Omega = 0.01
# interval for spectroscopy
Delta_interval = 0.2
# number of scan points
dimdelta = 30
# Detunings vector
Deltavet = np.linspace(-Delta_interval, +Delta_interval, dimdelta)
# excited state population
excited_state_p = 0*Deltavet

for n in range(dimdelta):
    # Setting each detuning different at each iteration
    argmap = {'Omega': Omega, 'Delta': Deltavet[n]}
    # Shroedinger Equation solver
    results = sesolve(Ham_t, fock(2,0), t, [rho1], args = argmap)
    # Grabbing excited state population
    excited_state_p[n] = results.expect[0][-1]
    print(n)



In [ ]:
fig, ax = plt.subplots()
ax.plot(Deltavet,excited_state_p);
ax.set_xlabel('detuning');
ax.set_ylabel('expectation value rho1');
ax.legend(("rho01"))

In [ ]:
# Rabi Hamiltonian
def Ham_t2(t, args):
    Omega = args['Omega']
    Delta = args['Delta']
    pulse = args['pulse']
    # laser interaction
    Hint = Qobj([[0, Omega], [Omega, 0]] )
    # laser detuning
    H0 = Qobj([[Delta, 0], [0, 0]] )

    if(t<pulse):
        Hret = H0+Hint
    else:
        Hret = H0


    return Hret

In [ ]:
# Calibration of pulses for Rabi spectroscopy.Making sure pulses are shorter than pi-pulses


t = np.linspace(0, 200, 50)
# Rabi frequency
Omega = np.array([0.01, 0.005])
# pulse lenght
pulse_length = np.array([75, 150])
# interval for spectroscopy
Delta_interval = 0.2
# number of scan points
dimdelta = 30
# Detunings vector
Deltavet = np.linspace(-Delta_interval, +Delta_interval, dimdelta)
# excited state population
excited_state_p = 0*Deltavet

print('1st pulse length = ', pulse_length[0])
print('2nd pulse length = ', pulse_length[1])

# Shroedinger Equation solver
argmap = {'Omega': Omega[0], 'Delta': 0.0, 'pulse' : pulse_length[0]}
results75 = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)
argmap = {'Omega': Omega[1], 'Delta': 0.0, 'pulse' : pulse_length[1]}
results150 = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)

fig, ax = plt.subplots()
ax.plot(t,results75.expect[0]);
ax.plot(t,results150.expect[0]);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho1 75", "rho1 150", "pulse"))



In [ ]:
# Ramsey Hamiltonian
def Ham_Ramsey(t, args):
    Omega = args['Omega']
    Delta = args['Delta']
    pulse_length = args['pulse_length']
    pulse_separation = args['pulse_separation']
    # laser interaction
    Hint = Qobj([[0, Omega], [Omega, 0]] )
    # laser detuning
    H0 = Qobj([[Delta, 0], [0, 0]] )

    if(t<pulse_length or (pulse_separation<t and t<pulse_separation+pulse_length )):
        Hret = H0+Hint
    else:
        Hret = H0


    return Hret

In [ ]:
#Rabi spectroscopy with different pulse duration

Delta_interval = 0.2
# number of scan points
dimdelta = 50
# Detunings vector
Deltavet = np.linspace(-Delta_interval, +Delta_interval, dimdelta)

excited_state_p = np.zeros((len(Omega), dimdelta))
for m in range(len(Omega)):
    for n in range(dimdelta):
        # Setting each detuning different at each iteration
        argmap = {'Omega': Omega[m], 'Delta': Deltavet[n], 'pulse': pulse_length[m]}
        # Shroedinger Equation solver
        results = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)
        # Grabbing excited state population
        excited_state_p[m,n] = results.expect[0][-1]
        print(n)

fig, ax = plt.subplots()
ax.plot(Deltavet,excited_state_p[0,:])
ax.plot(Deltavet,excited_state_p[1,:])
ax.set_xlabel('detuning');
ax.set_ylabel('expectation value rho1');
ax.legend(("rho1 75", "rho1 150"))

In [ ]:
# How to use a curve as a probability function and extract average and standard deviation

# making of the curve a probability distribution
prob_distr = excited_state_p
for m in range(len(Omega)):
    # calculating normalization
    norm = np.sum(excited_state_p[m,:])
    # normalizing every element
    prob_distr[m,:] = excited_state_p[m,:]/norm

print('normalized 75? ', np.sum(prob_distr[0,:]))
print('normalized 150? ', np.sum(prob_distr[1,:]))

Delta_avg = 0*Omega
Delta_avg2 = 0*Omega
sigma = 0*Omega
for m in range(len(Omega)):
    # average value calculated as  avg = sum ( p(x) * x)
    Delta_avg[m] = np.sum(np.multiply(prob_distr[m,:], Deltavet))
    # average value of the square calculated as  avg = sum ( p(x) * x^2)
    Delta_avg2[m] = np.sum(np.multiply(prob_distr[m,:], Deltavet**2))
    # standard deviation considering that    variance = <x^2>-<x>^2
    sigma = np.sqrt(Delta_avg2 - Delta_avg**2)
    print('average', Delta_avg[m])
    print('sigma', Delta_avg2[m])

fig, ax = plt.subplots()
ax.plot(Deltavet,prob_distr[0,:])
ax.plot(Deltavet,prob_distr[1,:])
ax.set_xlabel('detuning');
ax.set_ylabel('probability');
ax.legend(("rho1 75", "rho1 150"))

In [ ]:
# Quantum trajectories with different pulses

t = np.linspace(0, 200, 50)
# Rabi frequency
Omega = np.array([0.1, 0.005])
# pulse lenght
pulse_length = np.array([7.85, 150])
# interval for spectroscopy
Delta_interval = 0.2
# number of scan points
dimdelta = 30
# Detunings vector
Deltavet = np.linspace(-Delta_interval, +Delta_interval, dimdelta)
# excited state population
excited_state_p = 0*Deltavet

print('1st pulse length = ', pulse_length[0])
print('2nd pulse length = ', pulse_length[1])

# Shroedinger Equation solver
argmap = {'Omega': Omega[0], 'Delta': 0.0, 'pulse' : pulse_length[0]}
results75 = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)
argmap = {'Omega': Omega[1], 'Delta': 0.0, 'pulse' : pulse_length[1]}
results150 = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)
argmap = {'Omega': Omega[0], 'Delta': 0.0, 'pulse_length' : pulse_length[0], 'pulse_separation' : pulse_length[1]}
resultsRamsey = sesolve(Ham_Ramsey, fock(2,0), t, [rho1], args = argmap)

fig, ax = plt.subplots()
ax.plot(t,results75.expect[0]);
ax.plot(t,results150.expect[0]);
ax.plot(t,resultsRamsey.expect[0]);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho1 75", "rho1 150", "Ramsey"))
print(results75.expect[0][-1])

In [ ]:
# Travel on the Bloch sphere for detuned Ramsey scheme

# Rabi frequency
Omega_Ramsey = 0.1
# pulse lenght
pulse_Rabi = 150
# calibrated pi/2-pulse
pihalf_pulse = 7.85

# time to simulate
t = np.linspace(0, 200, 512)

# operators we would like to know the expectation value
e_ops = [sigmax(), sigmay(), sigmaz(), rho0, rho1]

argmap = {'Omega': Omega_Ramsey, 'Delta': 0.02,\
          'pulse_length' : pihalf_pulse, 'pulse_separation' : pulse_Rabi}

# Shroedinger Equation solver
results = sesolve(Ham_Ramsey, fock(2,0), t, e_ops, args = argmap)

fig, ax = plt.subplots()
ax.plot(t,results.expect[3]);
ax.plot(t,results.expect[4]);
ax.set_xlabel('time');
ax.set_ylabel('expectation value rho0');
ax.legend(("rho0", "rho1", "pulse"))

# Showing on the Bloch sphere
b = Bloch()
pnt = [results.expect[0],results.expect[1],results.expect[2]]
b.add_points(pnt)
b.show()


In [ ]:
# RAbi vs RAmsey

t = np.linspace(0, 100, 100)
Delta_interval = 0.2
# number of scan points
dimdelta = 50
# Detunings vector
Deltavet = np.linspace(-Delta_interval, +Delta_interval, dimdelta)

excited_state_Rabi = 0*Deltavet
excited_state_Ramsey = 0*Deltavet
# Rabi frequency
Omega_Rabi = 0.005
Omega_Ramsey = 0.1
# pulse lenght
pulse_Rabi = 150
pihalf_pulse = 7.85
#------------------------------------------------
# RABI
# Setting each detuning different at each iteration

for n in range(dimdelta):
    argmap = {'Omega': Omega_Rabi, 'Delta': Deltavet[n], 'pulse': pulse_Rabi}
    # Shroedinger Equation solver
    results = sesolve(Ham_t2, fock(2,0), t, [rho1], args = argmap)
    # Grabbing excited state population
    excited_state_Rabi[n] = results.expect[0][-1]
    print(n)

#------------------------------------------------
# RAMSEY
for n in range(dimdelta):
    argmap = {'Omega': Omega_Ramsey, 'Delta': Deltavet[n], 'pulse_length' : pihalf_pulse, 'pulse_separation' : pulse_Rabi}
    # Shroedinger Equation solver
    results = sesolve(Ham_Ramsey, fock(2,0), t, [rho1], args = argmap)
    # Grabbing excited state population
    excited_state_Ramsey[n] = results.expect[0][-1]
    print(n)

#------------------------------------------------

fig, ax = plt.subplots()
ax.plot(Deltavet,excited_state_Rabi)
ax.plot(Deltavet,excited_state_Ramsey)
ax.set_xlabel('detuning');
ax.set_ylabel('expectation value rho1');
ax.legend(("rho1 75", "rho1 150"))